<a href="https://colab.research.google.com/github/zhh210/flea_market/blob/master/Nested.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torchtext, spacy, torch

In [0]:
e# set up fields
TEXT = torchtext.data.Field(lower=True, include_lengths=True, batch_first=True)
LABEL = torchtext.data.LabelField(dtype=torch.float)
# make splits for data
train_d, test_d = torchtext.datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 37.8MB/s]


In [0]:
# ' '.join(train[0].text)
import numpy as np
np.urandom.rand()

0.1646868341335962

In [0]:
nlp = spacy.load('en')

def tokenizer(input):
    # Tokenize a sentence
    return [x.text for x in nlp.tokenizer(input) if x.text != " "]

NESTED_TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer)
TEXT = torchtext.data.NestedField(NESTED_TEXT)
LABEL = torchtext.data.LabelField(dtype=torch.float)
fields = [('text',TEXT),('label',LABEL)]
import numpy as np
examples = [torchtext.data.Example.fromlist([[j.text for j in nlp(' '.join(i.text)).sents], i.label],fields) for i in test_d if np.random.rand()<2]
test_data = torchtext.data.Dataset(examples, fields)
examples = [torchtext.data.Example.fromlist([[j.text for j in nlp(' '.join(i.text)).sents], i.label],fields) for i in train_d]
train_data = torchtext.data.Dataset(examples, fields)
train_data[0].text

[['i',
  "'ve",
  'seen',
  'this',
  'movie',
  'at',
  'least',
  'fifty',
  'times',
  'and',
  'after',
  'watching',
  'it',
  'last',
  'week',
  'for',
  'the',
  'first',
  'time',
  'in',
  'a',
  'long',
  'time',
  'i',
  'still',
  'felt',
  'it.<br',
  '/><br'],
 ['/>the',
  'story',
  'itself',
  'was',
  'incredible',
  'but',
  'came',
  'alive',
  'by',
  'spielberg',
  "'s",
  'expertise',
  'and',
  'the',
  'fabulous',
  'cast',
  'including',
  'whoopi',
  'goldberg',
  ',',
  'oprah',
  'winfrey',
  ',',
  'danny',
  'glover',
  ',',
  'and',
  'margaret',
  'avery',
  '.'],
 ['akosua',
  'busia',
  'deserved',
  'an',
  'oscar',
  'nomination',
  'for',
  'her',
  'short',
  'but',
  'powerful',
  'portrayal',
  'of',
  'nettie.<br',
  '/><br',
  "/>you'll",
  'experience',
  'every',
  'human',
  'emotion',
  'while',
  'watching',
  'this',
  'film',
  '.'],
 ['i', 'laughed', ',', 'cried', ',', 'and', 'got', 'angry', '.'],
 ['like',
  'most',
  'great',
  'movi

In [0]:
NESTED_TEXT.build_vocab(train_data,min_freq=1,vectors="glove.6B.200d")
LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [02:42, 5.30MB/s]                           
100%|█████████▉| 399372/400000 [00:37<00:00, 10665.31it/s]

In [0]:
import torch
BATCH_SIZE = 10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator,test_iterator = torchtext.data.BucketIterator.splits(
    (train_data, test_data), 
    batch_size=BATCH_SIZE, 
    device=device,sort_key=lambda x: len(x.text))
next(iter(test_iterator))


[torchtext.data.batch.Batch of size 10]
	[.text]:[torch.cuda.LongTensor of size 10x1x147 (GPU 0)]
	[.label]:[torch.cuda.FloatTensor of size 10 (GPU 0)]

In [0]:
tmp = next(iter(test_iterator))
for i in range(tmp.text.shape[1]):print(tmp.text[:,i,:].shape)
tmp.label

torch.Size([10, 147])


tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0')

In [0]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, 
                 output_dim, n_layers, bidirectional, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, 
                           bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        hiddens = []
        
        for i in range(text.shape[1]):
            texti = text[:,i,:].transpose(dim0=1,dim1=0)
            embedded = self.dropout(self.embedding(texti))
        
        #embedded = [sent len, batch size, emb dim]
        
            output, (hidden, cell) = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim * num directions]
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
            hiddens.append(hidden)
                
        #hidden = [batch size, hid dim * num directions]
#         print(embedded.shape, hidden.shape, torch.mean(torch.stack(hiddens),dim=0).shape)
#         import pdb; pdb.set_trace()
        return self.fc(torch.mean(torch.stack(hiddens),dim=0).squeeze(0))

In [0]:
INPUT_DIM = len(NESTED_TEXT.vocab)
EMBEDDING_DIM = 200
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)

import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc
  
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label.float().squeeze(0))
        
        acc = binary_accuracy(predictions, batch.label.float().squeeze(0))
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)
  
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label.float().squeeze(0))
            
            acc = binary_accuracy(predictions, batch.label.float().squeeze(0))

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
N_EPOCHS = 1

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, test_iterator, criterion)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')

100%|█████████▉| 399372/400000 [00:51<00:00, 10665.31it/s]

| Epoch: 01 | Train Loss: 0.694 | Train Acc: 51.48% | Val. Loss: 0.693 | Val. Acc: 50.44% |


In [0]:
for i in train_data.label: print(i)

In [0]:
LABEL

In [0]:
LABEL.vocab.stoi

In [0]:
10%3